In [1]:
!pip install langchain faiss-cpu tavily-python groq
!pip install python-dotenv
!pip install langchain-community

In [2]:
import os
os.environ["GROQ_API_KEY"] = "insert your key here"
os.environ["TAVILY_API_KEY"] = "insert your key here"

In [3]:
from langchain_community.document_loaders import TextLoader
import glob

In [4]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir("/content/drive/My Drive")
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 churn-bigml-20.csv			    'Loan Prediction.csv'
 climate_2009_2016.csv			     Mall_Customers.csv
'Colab Notebooks'			     model.pkl
 crew_data				     models
 Cuisine_rating.csv			     movies.pkl
'Data_Science_Week_8_Assignment (3).ipynb'   similarity.pkl
 df_original.csv			     streamlit_app
 df_scaled.csv				     tmdb_5000_credits.csv
 Diabetes-Classification.csv		     tmdb_5000_movies.csv
 fitness_class_2212.csv			     twitter_training.csv
 Foreign_Exchange_Rates.xls		     vectorstore.pkl
 heart.csv				     weather_data_extended.csv
 IRIS.csv				    'Week 8.ipynb'


In [5]:
docs = []

#load all txt files from crew_data
for file in glob.glob('/content/drive/My Drive/crew_data/*.txt'):
    docs.extend(TextLoader(file).load())

# check loaded docs
print(f"Loaded {len(docs)} documents")

Loaded 2 documents


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

print(f"Created {len(chunks)} chunks")

Created 1 chunks


In [7]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings

In [8]:
os.environ["OPENAI_API_KEY"] = "insert your key here"  # must have API key

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)

/tmp/ipython-input-4022086550.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-openai package and should be used instead. To use it run `pip install -U `langchain-openai` and import as `from `langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
#I have run out of 'quota' in OpenAI, so I will just show the rest of the code that will work if this quote is met
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI  # Or Groq model if you switch

#use the FAISS vectorstore as retriever
retriever = vectorstore.as_retriever()

#set up QA chain
llm = ChatOpenAI(temperature=0)  # or GroqChat model if available
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [ ]:
from tavily import TavilyClient
tavily_client = TavilyClient(api_key="insert your key here")
results = tavily_client.search("latest advancements in LLMs")

In [ ]:
# Researcher: gets info from vectorstore + Tavily
research_results = retriever.get_relevant_documents("RAG explanation")

# Writer: generates content
writer_output = llm.generate_text("Write a report based on:", research_results)

# Critic: evaluates
critic_feedback = llm.generate_text("Check for accuracy and coherence:", writer_output)

In [ ]:
# Example query through QA chain
print(qa_chain.run("Explain RAG in simple terms."))

In [ ]:
#Save the vectorstore and model
import pickle

# Save FAISS index & embeddings for later
with open("vectorstore.pkl", "wb") as f:
    pickle.dump(vectorstore, f)


#can load with

with open("vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)